## 1. Database Preprocessing
For data preprocessing, Pandas and Sqlite3 are used to extract data from sql files and perform data preprocessing including dealing with missing values etc.

In [1]:
import pandas as pd
import sqlite3

def select_return_table(table_name):
    # Select from all records and convert to pandas dataframe
    data = curs.execute('SELECT * FROM %s' % table_name).fetchall()
    column = [element[1] for element in curs.execute('PRAGMA table_info(%s)' % table_name).fetchall()]
    return pd.DataFrame(data, columns=column)

def get_missing_value_perc(df, cond=lambda x: x == 'null'):
    # Check missing value and output percentage
    df_sum = df.applymap(cond).sum()
    df_percentage = df.applymap(cond).sum() / df.applymap(lambda data: data == 'null').count()
    df_percentage = df_percentage.apply(lambda x: '{0:.2f}%'.format(x * 100))
    return pd.concat([df_sum, df_percentage], axis=1, keys=['Missing Value', 'Missing Value (%)'])

## 1.1 Database Connection
Five tables were created during the crawling stage, containing information about race, horse, individual past result (of trainer, jockey, breeder and owner), trainer and jockey profiles. Those sql files can be easily transformed into Pandas dataframe for further processing.

In [2]:
# Establish database connection and check table name
conn = sqlite3.connect('temp/race.db')
curs = conn.cursor()
table_name = curs.execute('SELECT name FROM sqlite_master WHERE type="table"').fetchall()
print(table_name)

[('race_record',), ('horse_record',), ('individual_record',), ('trainer_profile',), ('jockey_profile',)]


In [3]:
# Read from record data
record_dict = {name[0]: select_return_table(name[0]) for name in table_name}
race_df = record_dict['race_record']
horse_df = record_dict['horse_record']
individual_df = record_dict['individual_record']
trainer_df = record_dict['trainer_profile']
jockey_df = record_dict['jockey_profile']

## 1.2 Integrity Check
This step is to check the integrity of the crawled data and confirm that there is little inconsistency with the actual data presented online as well as some basic rules for national horse racing events in Japan (e.g. there should be 12 races for one place). More details are in <a href='http://www.jra.go.jp/'> JRA (Japan Racing Association) webpage</a>.

### 1.2.1 Race Record
The following shows the attributes in race record table as well as some basic statistics. Grouping race records by run data, place and race number yields the total numebr of race happening in one place on a certain date.

In [4]:
# Snapshot of tha race_record dataframe
print(race_df.shape)
race_df.describe().T

(380733, 27)


,count,unique,top,freq
run_date,380733,829,2007-12-22,560
place,380733,10,中山,55075
race,380733,12,12R,33244
title,380733,1766,3歳未勝利,89501
type,380733,3,ダ,184679
track,380733,4,右,250855
distance,380733,71,1200m,88303
weather,380733,6,晴,229543
condition,380733,4,良,295261
time,380733,90,16:20,10478


The following proves that it is thus true that around 12 races are held for a single place for national racing events in Japan. A sample of 10 racing events are shown below.

In [5]:
# Ensure that (almost) all races on the same day at the same place have a count of 12
race_count = curs.execute('SELECT DISTINCT run_date, place, race from race_record').fetchall()
race_count_df = pd.DataFrame(race_count, columns=['run_date', 'place', 'race'])
race_count_df.groupby(['run_date', 'place']).count().sample(n=10)

,,race
run_date,place,
2006-04-16,福島,12
2001-10-14,東京,12
2004-06-12,東京,12
2000-07-09,阪神,12
2001-11-10,東京,12
2006-12-09,中山,12
2003-12-20,中京,12
2000-04-09,阪神,12
2005-03-05,中京,12


### 1.2.2 Horse Record
Similar check is done for horse record.

In [6]:
# Check the data columns
horse_df.sample(n=3)

,horse_name,date_of_birth,trainer,owner,breeder,place_of_birth,transaction_price,prize_obtained,race_record,highlight_race,relatives,parents,status,gender,breed,offer_info
7493,マルブツロドリゴ,1994年5月13日,坪正直 (栗東),大沢毅,中央牧場,新冠町,-,"5,310万円 (中央)",27戦4勝 [ 4-2-3-18 ],99'醍醐特別(900万下),マルブツホープ 、 ジェネラスレディ,ロドリゴデトリアーノ マルブツロンリー,抹消,牡,栗毛,null
2645,ゴールデンストーク,1995年6月14日,森安弘昭 (美浦),高橋秀昌,高橋農場,伊達市,-,60万円 (中央) /150万円 (地方),20戦0勝 [ 0-1-1-18 ],,ゴールデンテンマ 、 トップアタック,リドヘイム バーバレラ,抹消,牝,青鹿毛,null
12228,メトロウイナー,1999年5月7日,坂田正行 (栗東),太田廣子,へいはた牧場,静内町,-,60万円 (中央),9戦0勝 [ 0-0-0-9 ],,チェルケッティ 、 レオポルディー,ロドリゴデトリアーノ シューシャ,抹消,牝,鹿毛,null


In [7]:
# Snapshot of tha race_record dataframe
print(horse_df.shape)
horse_df.describe().T

(40335, 16)


,count,unique,top,freq
horse_name,40335,40335,スナークハンター,1
date_of_birth,40335,2136,2002年4月3日,72
trainer,40335,1533,和田正道 (美浦),186
owner,40335,4980,社台レースホース,616
breeder,40335,3296,社台ファーム,1518
place_of_birth,40335,93,浦河町,7702
transaction_price,40335,3492,-,33686
prize_obtained,40335,18439,0万円,6476
race_record,40335,14251,2戦0勝 [ 0-0-0-2 ],1453
highlight_race,40335,8077,,12872


### 1.2.3 Individual Record
Similar check is done for individual record. It provides yearly consolidated results from each individual related to horse racing events.

In [8]:
# Check the data columns
individual_df.sample(n=3)

,individual_type,name,year,rank,first,second,third,out,races_major,wins_major,...,wins_flat,races_grass,wins_grass,races_dirt,wins_dirt,wins_percent,wins_percent_2nd,wins_percent_3rd,prize_obtained,representative_horse
7334,騎手,村山明,1994,51,21,18,29,156,5,1,...,15,99,9,91,6,.094,.174,.304,"34,328.7",キンセンアラシ
4766,騎手,常石勝義,1996,81,12,15,6,169,1,0,...,12,95,6,108,6,.059,.133,.163,"18,353.5",エイティグロー
13373,馬主,安田修,1988,751,2,0,1,13,1,0,...,2,8,1,8,1,.125,.125,.188,"1,278.0",シンコウサムソン


In [9]:
# Snapshot of tha race_record dataframe
print(individual_df.shape)
individual_df.describe().T

(88111, 23)


,count,unique,top,freq
individual_type,88111,4,馬主,37158
name,88111,7985,浅川皓司,33
year,88111,33,2000,4160
rank,88111,1499,1313,768
first,88111,192,0,36237
second,88111,166,0,38074
third,88111,154,0,36454
out,88111,752,1,6451
races_major,88111,126,0,58414
wins_major,88111,31,0,81477


### 1.2.4 Trainer Profile
Similar check is done for trainer profiles. It lists personal information for a certain trainer.

In [10]:
# Check the data columns
trainer_df.sample(n=3)

,trainer_name,date_of_birth,place_of_birth,first_run_date,first_run_horse,first_win_date,first_win_horse
663,[地]黒田隆男,1957/11/04,null,null,null,null,null
747,[地]山田和久,1968/05/23,null,null,null,null,null
553,[東]本間忍,1957/07/28,北海道,2002/03/16,バトルステージ,2002/07/14,ビバリーホリデー


In [11]:
# Snapshot of tha race_record dataframe
print(trainer_df.shape)
trainer_df.describe().T

(839, 7)


,count,unique,top,freq
trainer_name,839,839,[地]浜田一夫,1
date_of_birth,839,810,1957/02/18,2
place_of_birth,839,36,null,617
first_run_date,839,154,null,619
first_run_horse,839,221,null,619
first_win_date,839,209,null,619
first_win_horse,839,221,null,619


### 1.2.5 Jockey Profile
Similar check is done for jockey profiles. It lists personal information for a certain jockey.

In [12]:
# Check the data columns
jockey_df.sample(n=3)

,jockey_name,date_of_birth,place_of_birth,blood_type,height,weight,first_flat_run_date,first_flat_run_horse,first_flat_win_date,first_flat_win_horse,first_obs_run_date,first_obs_run_horse,first_obs_win_date,first_obs_win_horse
263,青木達彦,1964/05/21,null,null,null,null,1987/09/20,ポールドヒューマ,null,null,null,null,null,null
531,ラスボン,1980/08/27,null,null,null,null,null,null,null,null,2004/03/27,ニコバリー,null,null
68,柴田未崎,1977/06/18,栃木県,B型,159cm,50kg,1996/03/02,ヒサゴシズカ,1996/06/16,マイネルダンケ,2003/04/06,サキノハマナス,2004/04/24,ハッピートウキョウ


In [13]:
# Snapshot of tha race_record dataframe
print(jockey_df.shape)
jockey_df.describe().T

(632, 14)


,count,unique,top,freq
jockey_name,632,632,菅原勲,1
date_of_birth,632,616,1971/03/12,2
place_of_birth,632,36,null,437
blood_type,632,5,null,439
height,632,23,null,437
weight,632,16,null,437
first_flat_run_date,632,338,null,33
first_flat_run_horse,632,592,null,33
first_flat_win_date,632,332,null,265
first_flat_win_horse,632,366,null,265


## 1.3 Preprocessing
The following shows further preprocessing of the dataset. Predominantly it resolves around dealing with missing values within each columns. As majority of the columns contain no missing values and some of them having over 90% of missing values or below 1%, simple dropping is performed for these columns.

### 1.3.1 Race Record

As the percentage of missing value for the latter columns is trivial in this case, it can be safely dropped without affecting the entire dataset.

In [14]:
# Check missing value
get_missing_value_perc(race_df)

,Missing Value,Missing Value (%)
run_date,0,0.00%
place,0,0.00%
race,0,0.00%
title,0,0.00%
type,0,0.00%
track,0,0.00%
distance,0,0.00%
weather,0,0.00%
condition,0,0.00%
time,0,0.00%


In [15]:
race_df = race_df.loc[race_df.applymap(lambda x: x == 'null').sum(axis=1) == 0, :]

### 1.3.2 Horse Record

The last column named 'offer_info' can simply be dropped from the dataset as it has 98.39% of missing values.

In [16]:
# Check missing value
get_missing_value_perc(horse_df)

,Missing Value,Missing Value (%)
horse_name,0,0.00%
date_of_birth,0,0.00%
trainer,0,0.00%
owner,0,0.00%
breeder,0,0.00%
place_of_birth,0,0.00%
transaction_price,0,0.00%
prize_obtained,0,0.00%
race_record,0,0.00%
highlight_race,0,0.00%


In [17]:
horse_df = horse_df.drop('offer_info', axis=1)

### 1.3.3 Individual Record

As individual records with missing value are all tied to solely one person, it could be simply dropped from the table.

In [18]:
# Check missing value
get_missing_value_perc(individual_df)

,Missing Value,Missing Value (%)
individual_type,0,0.00%
name,0,0.00%
year,0,0.00%
rank,0,0.00%
first,63,0.07%
second,63,0.07%
third,63,0.07%
out,63,0.07%
races_major,63,0.07%
wins_major,63,0.07%


In [19]:
individual_df.loc[individual_df.applymap(lambda x: x == 'null').sum(axis=1) > 0]['name'].value_counts(ascending=False)[:5]

山本功              1
門別賢一             1
杉元四男             1
Mr Jakob Rohr    1
加藤泰章             1
Name: name, dtype: int64

In [20]:
individual_df = individual_df.loc[individual_df.applymap(lambda x: x == 'null').sum(axis=1) == 0, :]

### 1.3.4 Trainer & Jockey Profile

Regarding place of birth, it is assumed that trainer/jockey without such as record would be treated as from outside Tokyo. With regards with other attributes, some further feature engineering is believed to be executable instead of treating them as missing values tentatively. It is suggestible that attributes such as first run date can be derived from the race record table despite not being shown here.

In [21]:
# Check missing value
get_missing_value_perc(trainer_df)

,Missing Value,Missing Value (%)
trainer_name,0,0.00%
date_of_birth,0,0.00%
place_of_birth,617,73.54%
first_run_date,619,73.78%
first_run_horse,619,73.78%
first_win_date,619,73.78%
first_win_horse,619,73.78%


In [22]:
# Check missing value
get_missing_value_perc(jockey_df)

,Missing Value,Missing Value (%)
jockey_name,0,0.00%
date_of_birth,0,0.00%
place_of_birth,437,69.15%
blood_type,439,69.46%
height,437,69.15%
weight,437,69.15%
first_flat_run_date,33,5.22%
first_flat_run_horse,33,5.22%
first_flat_win_date,265,41.93%
first_flat_win_horse,265,41.93%


In [23]:
trainer_df['place_of_birth'] = trainer_df['place_of_birth'].apply(lambda x: x if x != 'null' else u'地方')
jockey_df['place_of_birth'] = jockey_df['place_of_birth'].apply(lambda x: x if x != 'null' else u'地方')

Finally, we can output the dataframes as csv files for further analysis.

In [24]:
race_df.to_csv('data/race.csv', encoding='utf-8')
horse_df.to_csv('data/horse.csv', encoding='utf-8')
individual_df.to_csv('data/individual.csv', encoding='utf-8')
trainer_df.to_csv('data/trainer.csv', encoding='utf-8')
jockey_df.to_csv('data/jockey.csv', encoding='utf-8')